In [1]:
import numpy as np
import pandas as pd
import wrds
import datetime
import pickle
import statsmodels.api as sm
import os
import json

## Define Key Functions 

In [387]:
def eventstudy(json_evtdata=None, evtdata = None, model='m', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df'): 

    # Set estimation, event and gap windows
    estwins = estwin + gap + np.abs(evtwins) # Estimation window start
    estwine = gap + np.abs(evtwins) + 1 # Estimation windown end
    evtwinx = estwins + 1 # this is to separate the pre-event and post-event dates within the evtwin, evt time value (0=event date, -10=window start, 10=window end)
    evtwins = np.abs(evtwins) # Convert the negative to positive 
    evtrang = evtwins + evtwine + 1 # total event window days
   
    df = conn.raw_sql(
        f"""
        SELECT 
            a.*,
            x.*,
            c.date as rdate, 
            c.ret as ret1, 
            f.mktrf,
            f.rf,
            (f.mktrf + f.rf) as mkt,
            f.smb, 
            f.hml, 
            f.umd,
            (1 + c.ret)*(coalesce(d.dlret, 0.00) +1) - 1 as ret, 
            (1 + c.ret)*(coalesce(d.dlret, 0.00) + 1)-1-(f.mktrf + f.rf) as exret_over_mkt,
            CASE WHEN c.date BETWEEN a.estwin1 AND a.estwin2 THEN 1 ELSE 0 END AS isest,
            CASE WHEN c.date BETWEEN a.evtwin1 AND a.evtwin2 THEN 1 ELSE 0 END AS isevt,
            CASE 
                WHEN c.date BETWEEN a.evtwin1 AND a.evtwin2 
                    THEN (RANK() OVER (PARTITION BY x.evtid ORDER BY c.date) - {evtwinx}) 
                    ELSE (RANK() OVER (PARTITION BY x.evtid ORDER BY c.date))
                END AS evttime, 
            CASE WHEN c.date = a.date THEN 1 ELSE 0 END AS evtflag
            
        FROM 
        (
            SELECT 
                date,
                lag(date, {estwins}) over (order by date) as estwin1, 
                lag(date, {estwine}) over (order by date) as estwin2, 
                lag(date, {evtwins}) over (order by date) as evtwin1, 
                lead(date, {evtwine}) over (order by date) as evtwin2
            FROM crsp_a_stock.dsi
        ) as a
        JOIN 
        (
            SELECT 
                to_char(x.edate::date, 'ddMONYYYY') || trim(to_char(x.permno, '9999999999')) AS evtid,
                x.permno,
                x.edate::date
            FROM 
                json_to_recordset('{json_evtdata}'::json) AS x(edate text, permno int)
        ) AS x
            ON a.date = x.edate
        JOIN 
            crsp_a_stock.dsf c
            ON x.permno = c.permno
            AND c.date BETWEEN a.estwin1 and a.evtwin2
        JOIN ff_all.factors_daily f
            ON c.date = f.date
        LEFT JOIN crsp_a_stock.dsedelist d
            ON x.permno = d.permno
            AND c.date = d.dlstdt
        WHERE f.mktrf IS NOT NULL
        AND c.ret IS NOT NULL
        ORDER BY x.evtid, x.permno, a.date, c.date    
        """
)

    # for each event, calcualte abnormal return based on chosen risk model. 

    for evt in evtdata:
        permno = pd.to_numeric(evt['permno'])
        xdate = evt['edate']
        edate = pd.to_datetime(xdate).date()
        df['edate'] = pd.to_datetime(df['edate']).dt.date
        
        ths_mask = (df['permno'] == permno) & (df['edate'] == edate)
        est_mask = (df['permno'] == permno) & (df['edate'] == edate) & (df['isest'] == 1)
        evt_mask = (df['permno'] == permno) & (df['edate'] == edate) & (df['isevt'] == 1)
        flg_mask = (df['permno'] == permno) & (df['edate'] == edate) & (df['evtflag'] == 1)   
    
    
        #########################################################
        # check to see if it meets the min obs for estimation window
        #########################################################
    
        # number of observation in the estimation window:
        _nobs_est = df[est_mask]['ret'].count()   
        # number of actual event date flags - 
        _flgs = df[flg_mask]['ret'].count()
        # number of obersations in the event window: 
        _wins_evt = df[evt_mask]['ret'].count()
        # Only carry out the analysis if the number of obsevations meets the minimum threshold
        if (_nobs_est >= minval) and (_flgs > 0) and (_wins_evt == evtrang):

         #########################################################
         # FF Regression model
         #########################################################

            # Market Model
            if model == 'm':
            
            # Set the variables from the output
                df_est = df[est_mask]
                _nobs = len(df_est[df_est.ret.notnull()])   # not null observations
    
             # Set Y to the estimation window records
                X = df_est["mktrf"]
                df_est = df_est.assign(**{'ret-rf': df_est['ret'] - df_est['rf']})
                Y= df_est['ret-rf']
    
                # Fit an OLS model with intercept on mktrf
                X = sm.add_constant(X)
                est = sm.OLS(Y, X).fit()         
    

                alpha = est.params.const
                beta_mkt = est.params.mktrf
    
                df.loc[evt_mask, 'INTERCEPT'] = alpha
                df.loc[evt_mask, 'alpha'] = alpha
                df.loc[evt_mask, 'beta_mkt'] = beta_mkt
                df.loc[evt_mask, 'RMSE'] = np.sqrt(est.mse_resid)
                df.loc[evt_mask, '_nobs'] = _nobs
                df.loc[evt_mask, 'var_estp'] = est.mse_resid
                df.loc[evt_mask, 'rsq'] = est.rsquared
               
    
                nloc = {'alpha': alpha, 'beta_mkt': beta_mkt, 'const': 0}
    
                def f_expret(row):
                    return (nloc['alpha'] + (nloc['beta_mkt'] * row['mktrf']))
                df.loc[evt_mask, 'expret'] = df[evt_mask].apply(f_expret, axis=1)
    
                def f_abret(row):
                    return (row['ret'] - row['expret'])
                df.loc[evt_mask, 'abret'] = df[evt_mask].apply(f_abret, axis=1)
    
                nloc = {'const': 0}
    
                def f_cret(row):
                    tmp = (1+nloc['const'])*(1+row['ret']) - 1
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'cret'] = df[evt_mask].apply(f_cret, axis=1)
                df.loc[evt_mask, 'cret_edate'] = nloc['const']
    
                nloc = {'const': 0}
    
                def f_cexpret(row):
                    tmp = (1+nloc['const'])*(1+row['expret']) - 1
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'cexpret'] = df[evt_mask].apply(f_cexpret, axis=1)
                df.loc[evt_mask, 'cexpret_edate'] = nloc['const']
    
                nloc = {'const': 0}
    
                def f_car(row):
                    tmp = (row['abret'] + nloc['const'])
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'car'] = df[evt_mask].apply(f_car, axis=1)
                df.loc[evt_mask, 'car_edate'] = nloc['const']
    
                nloc = {'const': 0}
    
                def f_sar(row):
                    tmp = (row['abret'] / np.sqrt(row['var_estp']))
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'sar'] = df[evt_mask].apply(f_sar, axis=1)
                df.loc[evt_mask, 'sar_edate'] = nloc['const']
    
                nloc = {'const': 0, 'evtrang': evtrang}
    
                def f_scar(row):
                    tmp = (row['car'] / np.sqrt((evtrang * row['var_estp'])))
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'scar'] = df[evt_mask].apply(f_scar, axis=1)
                df.loc[evt_mask, 'scar_edate'] = nloc['const']
    
                nloc = {'const': 0}
    
                def f_bhar(row):
                    tmp = (row['cret'] - row['cexpret'])
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'bhar'] = df[evt_mask].apply(f_bhar, axis=1)
                df.loc[evt_mask, 'bhar_edate'] = nloc['const']
    
            # Fama-French model: 
            elif model == "ff": 
                df_est = df[est_mask]
                X = df_est[['smb', 'hml', 'mktrf']]
                df_est = df_est.assign(**{'ret-rf': df_est['ret'] - df_est['rf']})
                Y= df_est['ret-rf']
            
                # Fit an OLS model with intercept on mktrf, smb, hml
                X = sm.add_constant(X)
                est = sm.OLS(Y,X).fit()
            
                alpha = est.params.const
                beta_mktrf = est.params.mktrf
                beta_smb = est.params.smb
                beta_hml = est.params.hml
            
                # Add the regression results to the df
                df.loc[evt_mask, 'INTERCEPT'] = alpha
                df.loc[evt_mask, 'alpha'] = alpha
                df.loc[evt_mask, 'beta_mktrf'] = beta_mktrf
                df.loc[evt_mask, 'beta_smb'] = beta_smb
                df.loc[evt_mask, 'beta_hml'] = beta_hml
                df.loc[evt_mask, 'RMSE'] = np.sqrt(est.mse_resid)
                df.loc[evt_mask, '_nobs'] = _nobs_est
                df.loc[evt_mask, 'var_estp'] = est.mse_resid
                df.loc[evt_mask, 'rsq'] = est.rsquared
                df.loc[evt_mask, '_p_'] = 2
                df.loc[evt_mask, '_edf_'] = len(Y) - 2
                
            
                params = {'alpha': alpha, 'beta_mktrf': beta_mktrf, 'beta_smb': beta_smb, 'beta_hml': beta_hml}
                # calculate expected returns based on the Famma-French Factors
                def f_expret(row): 
                    return params['alpha'] + (params['beta_mktrf'] * row['mktrf']) + (params['beta_smb'] * row['smb']) + (params['beta_hml'] * row['hml'])+ row['rf']
                df.loc[evt_mask, 'expret'] = df[evt_mask].apply(f_expret, axis = 1)
                
                # calcualte abnormal return using the actual return minus the expected return
                def f_abret(row):
                    return (row['ret'] - row['expret'])
                df.loc[evt_mask, 'abret'] = df[evt_mask].apply(f_abret, axis = 1)
                
                
                nloc = {'const': 0}
                # calculate cumulative returns during the evtwin
                def f_cret(row):
                    tmp = (1+nloc['const'])*(1+row['ret']) - 1
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'cret'] = df[evt_mask].apply(f_cret, axis = 1)
                df.loc[evt_mask, 'cret_edate'] = nloc['const']
                
                
                nloc = {'const': 0}
                # calculate cumulative expected returns
                def f_cexpret(row):
                    tmp = (1+nloc['const'])*(1+row['expret']) - 1
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'cexpret'] = df[evt_mask].apply(f_cexpret, axis=1)
                df.loc[evt_mask, 'cexpret_edate'] = nloc['const']
                
                
                nloc = {'const': 0}
                # calculate cumulative abnormal returns
                def f_car(row): 
                    tmp = (row['abret'] + nloc['const'])
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'car'] = df[evt_mask].apply(f_car, axis=1)
                df.loc[evt_mask, 'car_edate'] = nloc['const']
                
                
                nloc = {'const': 0}
                # calculate standardized abnormal returns: The standardization of ARs dampens the effect of stocks with high variances on further statistical tests
                def f_sar(row):
                    tmp = (row['abret'] / np.sqrt(row['var_estp']))
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'sar'] = df[evt_mask].apply(f_sar, axis=1)
                df.loc[evt_mask, 'sar_edate'] = nloc['const']
                
                
                nloc = {'const': 0, 'evtrang': evtrang}
                # calculate standardized cumulative abnormal returns
                def f_scar(row):
                    tmp = (row['car'] / np.sqrt((evtrang * row['var_estp'])))
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'scar'] = df[evt_mask].apply(f_scar, axis=1)
                df.loc[evt_mask, 'scar_edate'] = nloc['const']
                
                
                nloc = {'const': 0}
                # calculate buy and hold abnormal returns:   
                def f_bhar(row):
                    tmp = (row['cret'] - row['cexpret'])
                    nloc['const'] = tmp
                    return tmp
                df.loc[evt_mask, 'bhar'] = df[evt_mask].apply(f_bhar, axis=1)
                df.loc[evt_mask, 'bhar_edate'] = nloc['const']
                
                # df.loc[evt_mask, 'pat_scale'] = (_nobs_est - 2.00) / (_nobs_est - 4.00)
                # df.loc[evt_mask, 'pat_scale_edate'] = (_nobs_est - 2.00) / (_nobs_est - 4.00)
                
    #################################
    #  STEP 4 - OUTPUT THE RESULTS  #
    #################################

    df_sta = df[df['isevt'] == 1]
    levt = df_sta['evttime'].unique()

    columns = [
        'evttime', 
        'ret_m',
        'expret_m',
        'abret_m',
        'cret_m',
        'cexpret_m',
        'abret_t',
        'car_m',
        'scar_m',
        'cexpret_edate_m',
        'cret_edate_m',
        'car_edate_m',
        'car_edate_t',
        'sar_t',
        'scar_edate_t',
        'bhar_edate_m']

    idxlist = list(levt)
    df_stats = pd.DataFrame(index=idxlist, columns=columns)
    # df_stats = df_stats.fillna(0.00000000).infer_objects(copy=False)  # with 0s rather than NaNs

    # Event
    df_stats['evttime'] = df_sta.groupby(['evttime'])['evttime'].unique()
    # Means
    df_stats['ret_m'] = df_sta.groupby(['evttime'])['ret'].mean()
    df_stats['bhar_m'] = df_sta.groupby(['evttime'])['bhar'].mean()
    df_stats['expret_m'] = df_sta.groupby(['evttime'])['expret'].mean()
    df_stats['abret_m'] = df_sta.groupby(['evttime'])['abret'].mean()
    df_stats['cret_m'] = df_sta.groupby(['evttime'])['cret'].mean()
    df_stats['cexpret_m'] = df_sta.groupby(['evttime'])['cexpret'].mean()
    df_stats['car_m'] = df_sta.groupby(['evttime'])['car'].mean()
    df_stats['cret_edate_m'] = df_sta.groupby(['evttime'])['cret_edate'].mean()
    df_stats['cexpret_edate_m'] = df_sta.groupby(['evttime'])['cexpret_edate'].mean()
    df_stats['car_edate_m'] = df_sta.groupby(['evttime'])['car_edate'].mean()
    df_stats['bhar_edate_m'] = df_sta.groupby(['evttime'])['bhar_edate'].mean()
    df_stats['sar_m'] = df_sta.groupby(['evttime'])['sar'].mean()
    df_stats['scar_m'] = df_sta.groupby(['evttime'])['scar'].mean()
    df_stats['scar_edate_m'] = df_sta.groupby(['evttime'])['scar_edate'].mean()
    
    
    # Standard deviations
    
    df_stats['abret_v'] = df_sta.groupby(['evttime'])['abret'].std()
    df_stats['sar_v'] = df_sta.groupby(['evttime'])['sar'].std()
    df_stats['car_v'] = df_sta.groupby(['evttime'])['car'].std()
    df_stats['scar_v'] = df_sta.groupby(['evttime'])['scar'].std()
    df_stats['car_edate_v'] = df_sta.groupby(['evttime'])['car_edate'].std()
    df_stats['scar_edate_v'] = df_sta.groupby(['evttime'])['scar_edate'].std()
    
    
    # Counts   
    df_stats['n'] = df_sta.groupby(['evttime'])['evttime'].count()

    # T statistics 1
    def tstat(row, m, v, n):
        return row[m] / (row[v] / np.sqrt(row[n]))

    df_stats['abret_t'] = df_stats.apply(tstat, axis=1, args=('abret_m', 'abret_v', 'n'))
    df_stats['sar_t'] = df_stats.apply(tstat, axis=1, args=('sar_m', 'sar_v', 'n'))
    df_stats['car_edate_t'] = df_stats.apply(tstat, axis=1, args=('car_edate_m', 'car_edate_v', 'n'))
    df_stats['scar_edate_t'] = df_stats.apply(tstat, axis=1, args=('scar_edate_m', 'scar_edate_v', 'n'))
    

    # Output file: 1. Last_Event_Day, 2. Event_Window_Stats

    ## Last Event Day : 
    if model == 'm':
        maxv = max(levt)
        df_evtd = df.loc[(df['isevt'] == 1) & (df['evttime'] == maxv), ['permno', 'edate', 'cret', 'cexpret', 'alpha', 'beta_mkt', 'car', 'bhar', 'scar']]
        df_evtd.sort_values(['permno', 'edate'], ascending=[True, True], inplace = True)

    elif model == 'ff': 
        maxv = max(levt)
        df_evtd = df.loc[(df['isevt'] == 1) & (df['evttime'] == maxv), ['permno', 'edate', 'cret', 'cexpret', 'alpha', 'beta_mktrf', 'beta_hml', 'beta_smb', 'car', 'bhar', 'scar']]
        df_evtd.sort_values(['permno', 'edate'], ascending=[True, True], inplace = True)

    ## Event Window:
    df_evtwin = df_stats[['evttime', 'ret_m', 'abret_m', 'sar_m', 'cret_m', 'bhar_m', 'car_m', 'scar_m', 'n', 'sar_t', 'abret_t']]

    if output == 'df': 
        retval = {}
        retval['evt_win_stats'] = df_evtwin
        retval['last_evt_day'] = df_evtd
        return retval

In [369]:
def get_evtdata(df, sector, evtdate):
    
    df = df[df['GICS Sector'] == sector].copy()
    df = df.assign(**{'edate': evtdate})
     
    data = df[['permno', 'edate']]
    evtdata = [
        {
            "permno": int(row['permno']),
            "edate": row['edate']
        } for _, row in data.iterrows()
    ]
    return evtdata

## Code Execution

In [375]:
# read in the sp500 files with GICS industry classification.
data = pd.read_csv('merged_sp500_df.csv')

# get energy companies within the sp500
evtdata_energy = get_evtdata(data, 'Energy', '11/09/2016')
json_evtdata_energy = json.dumps(evtdata_energy).replace("'", "''")
# get financial companies within the sp500
evtdata_financials = get_evtdata(data, 'Financials', '11/09/2016')
json_evtdata_financials = json.dumps(evtdata_financials).replace("'", "''")
# get technology companies within the sp500
evtdata_technology = get_evtdata(data, 'Information Technology', '11/09/2016')
json_evtdata_technology = json.dumps(evtdata_technology).replace("'", "''")

In [281]:
data['GICS Sector'].unique()

array(['Health Care', 'Materials', 'Industrials',
       'Consumer Discretionary', 'Information Technology', 'Financials',
       'Consumer Staples', 'Utilities', 'Real Estate', 'Energy',
       'Communication Services'], dtype=object)

In [388]:
# get results for each sector
## energy
ff = {}
m = {}

ff['energy'] = eventstudy(json_evtdata=json_evtdata_energy, evtdata = evtdata_energy, model='ff', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')
ff['financials'] = eventstudy(json_evtdata=json_evtdata_financials, evtdata = evtdata_financials, model='ff', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')
ff['technology'] = eventstudy(json_evtdata=json_evtdata_technology, evtdata = evtdata_technology, model='ff', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')

m['energy'] = eventstudy(json_evtdata=json_evtdata_energy, evtdata = evtdata_energy, model='m', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')
m['financials'] = eventstudy(json_evtdata=json_evtdata_financials, evtdata = evtdata_financials, model='m', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')
m['technology'] = eventstudy(json_evtdata=json_evtdata_technology, evtdata = evtdata_technology, model='m', estwin=100, gap=50, evtwins=-10, evtwine=10, minval=70, output='df')

# combine all the model results into a summary dictionary
Summary_Result = {'ff_model': ff, 'm_model': m}

In [389]:
# aggregate the output information for last_event_day and comparing different sectors for _ff and _m models

for key in Summary_Result:
    for stats in Summary_Result[key]:
        if key == 'ff_model':
            Aggregate_Output[f'last_event_day_ff_{stats}'] = Summary_Result[key][stats]['last_evt_day'][['cret', 'cexpret', 'alpha', 'beta_mktrf', 'beta_hml',
           'beta_smb', 'car', 'bhar', 'scar']].mean()
        if key == "m_model": 
            Aggregate_Output[f'last_event_day_m_{stats}'] = Summary_Result[key][stats]['last_evt_day'][['cret', 'cexpret', 'alpha', 'beta_mkt', 'car', 'bhar', 'scar']].mean()

## Model Output Examples

In [390]:
pd.DataFrame(Aggregate_Output).T

,alpha,beta_hml,beta_mkt,beta_mktrf,beta_smb,bhar,car,cexpret,cret,scar
last_event_day_ff_energy,0.001323,1.658683,NaN,1.034413,0.535389,-0.164807,-0.134417,0.245854,0.081047,-1.346752
last_event_day_ff_financials,-0.000235,0.596087,NaN,1.360302,0.087420,0.009461,0.009501,0.107432,0.116893,0.346074
last_event_day_ff_technology,-0.000398,0.126856,NaN,1.260922,0.595145,-0.035154,-0.030405,0.090752,0.055599,-0.357670
last_event_day_m_energy,0.001035,NaN,1.499534,NaN,NaN,-0.009767,-0.004259,0.090814,0.081047,-0.020398
last_event_day_m_financials,-0.000390,NaN,1.500854,NaN,NaN,0.059034,0.053306,0.057859,0.116893,1.098993
last_event_day_m_technology,-0.000149,NaN,1.436916,NaN,NaN,-0.005485,-0.003865,0.061084,0.055599,-0.085911


In [391]:
Summary_Result['ff_model']['financials']['last_evt_day']

,permno,edate,cret,cexpret,alpha,beta_mktrf,beta_hml,beta_smb,car,bhar,scar
170,10138,2016-11-09,0.124775,0.043119,-0.001718,1.425454,0.289655,-0.068424,0.079083,0.081656,2.480135
341,10696,2016-11-09,0.079022,-0.031133,-0.000488,1.016480,-0.671347,-0.351677,0.108506,0.110155,3.089415
512,11990,2016-11-09,0.056902,0.135369,-0.000735,1.137291,0.678831,0.923586,-0.069853,-0.078467,-1.007548
683,12449,2016-11-09,-0.113649,0.138773,0.000179,1.158303,0.709563,0.533849,-0.242632,-0.252422,-6.121169
854,14601,2016-11-09,0.357416,0.228230,-0.000360,1.291192,1.145678,1.591092,0.112377,0.129187,1.842668
...,...,...,...,...,...,...,...,...,...,...,...
13337,92108,2016-11-09,0.057414,0.138317,-0.001187,1.454149,0.646569,0.945165,-0.072974,-0.080903,-1.249340
13508,92121,2016-11-09,0.222757,0.141618,0.000647,1.024685,0.808868,0.360848,0.071712,0.081139,1.517762
13679,92402,2016-11-09,-0.032761,0.069654,0.001003,1.151850,-0.076957,0.039738,-0.099185,-0.102415,-3.085691
13850,92611,2016-11-09,-0.027990,0.010263,-0.000173,1.274578,-0.480289,-0.142256,-0.037199,-0.038253,-1.022113


In [392]:
Summary_Result['ff_model']['energy']['evt_win_stats']

,evttime,ret_m,abret_m,sar_m,cret_m,bhar_m,car_m,scar_m,n,sar_t,abret_t
-10,[-10],-0.000138,-0.006205,-0.272864,-0.000138,-0.006205,-0.006205,-0.059544,30,-2.039240,-1.891308
-9,[-9],-0.003693,-0.005761,-0.329990,-0.004020,-0.012184,-0.011966,-0.131553,30,-1.348907,-1.180468
-8,[-8],-0.008590,-0.007715,-0.294143,-0.012569,-0.019866,-0.019681,-0.195741,30,-0.987201,-1.627111
-7,[-7],-0.014335,-0.017854,-0.858631,-0.026860,-0.037721,-0.037536,-0.383109,30,-4.161136,-4.419089
-6,[-6],-0.003785,0.000633,0.067311,-0.030492,-0.036919,-0.036903,-0.368421,30,0.266205,0.132648
-5,[-5],-0.015651,-0.010099,-0.512477,-0.045933,-0.046819,-0.047002,-0.480252,30,-2.947325,-2.438098
-4,[-4],0.009740,-0.002408,-0.148163,-0.037044,-0.050174,-0.049410,-0.512584,30,-0.728303,-0.495391
-3,[-3],-0.004137,-0.001588,-0.083678,-0.041085,-0.051649,-0.050998,-0.530844,30,-0.599334,-0.514365
-2,[-2],0.023408,0.000147,0.011743,-0.018805,-0.052870,-0.050851,-0.528282,30,0.097359,0.056113
-1,[-1],-0.001538,-0.002931,-0.102301,-0.020098,-0.055618,-0.053782,-0.550605,30,-0.720852,-0.795164


In [393]:
Summary_Result['m_model']['financials']['evt_win_stats']

,evttime,ret_m,abret_m,sar_m,cret_m,bhar_m,car_m,scar_m,n,sar_t,abret_t
-10,[-10],0.003425,0.007267,0.735487,0.003425,0.007267,0.007267,0.160496,82,4.566598,4.491009
-9,[-9],-0.002458,0.002885,0.217439,0.000999,0.010159,0.010152,0.207945,82,1.291972,1.742613
-8,[-8],-0.002460,0.002282,0.349780,-0.001459,0.012391,0.012434,0.284274,82,2.742820,1.588144
-7,[-7],0.000134,0.000224,0.085529,-0.001376,0.012559,0.012657,0.302938,82,0.670283,0.190859
-6,[-6],-0.004849,0.005746,0.457240,-0.006229,0.018127,0.018404,0.402716,82,3.668596,4.213177
-5,[-5],-0.009178,0.002168,0.153456,-0.015486,0.019894,0.020572,0.436203,82,0.915733,1.189943
-4,[-4],0.002991,0.009384,0.970724,-0.012515,0.028993,0.029956,0.648032,82,7.951191,6.823923
-3,[-3],-0.003721,-0.001530,-0.157810,-0.016231,0.027356,0.028426,0.613595,82,-1.478774,-1.416533
-2,[-2],0.021642,-0.011437,-1.037318,0.005176,0.017330,0.016989,0.387234,82,-9.356261,-9.126268
-1,[-1],0.001771,-0.003843,-0.265128,0.006930,0.013544,0.013147,0.329378,82,-2.448991,-3.663759


In [222]:
X1 = df_12476[['smb', 'hml', 'mktrf']]
df_12476['ret-rf'] = df_12476['ret'] - df_12476['rf']
Y1= df_12476['ret-rf']

# Fit an OLS model with intercept on mktrf, smb, hml
X1 = sm.add_constant(X1)
est = sm.OLS(Y1,X1).fit()
est.summary()

C:\Users\m1852\AppData\Local\Temp\ipykernel_5544\2893328115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_12476['ret-rf'] = df_12476['ret'] - df_12476['rf']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ret-rf   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     14.42
Date:                Tue, 24 Sep 2024   Prob (F-statistic):           7.93e-08
Time:                        00:55:40   Log-Likelihood:                 226.88
No. Observations:                 100   AIC:                            -445.8
Df Residuals:                      96   BIC:                            -435.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0043      0.003      1.668      0.099      -0.001       0.009
smb            0.7185      0.638      1.126      0.263      -0.548       1.985
hml            2.3489      0.499      4.707      0.000       1.358       3.339
mktrf          0.8668      0.390      2.221      0.029       0.092       1.641
==============================================================================
Omnibus:                        0.723   Durbin-Watson:                   2.202
Prob(Omnibus):                  0.697   Jarque-Bera (JB):                0.307
Skew:                           0.061   Prob(JB):                        0.858
Kurtosis:                       3.242   Cond. No.                         274.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [221]:
X = df_12476[['mktrf']]
df_12476['ret-rf'] = df_12476['ret'] - df_12476['rf']
Y= df_12476['ret-rf']

# Fit an OLS model with intercept on mktrf, smb, hml
X = sm.add_constant(X)
est = sm.OLS(Y,X).fit()
est.summary()

C:\Users\m1852\AppData\Local\Temp\ipykernel_5544\2949152538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_12476['ret-rf'] = df_12476['ret'] - df_12476['rf']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ret-rf   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     17.49
Date:                Tue, 24 Sep 2024   Prob (F-statistic):           6.29e-05
Time:                        00:53:54   Log-Likelihood:                 216.50
No. Observations:                 100   AIC:                            -429.0
Df Residuals:                      98   BIC:                            -423.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0039      0.003      1.380      0.171      -0.002       0.010
mktrf          1.5154      0.362      4.182      0.000       0.796       2.234
==============================================================================
Omnibus:                        8.658   Durbin-Watson:                   2.172
Prob(Omnibus):                  0.013   Jarque-Bera (JB):               10.133
Skew:                           0.480   Prob(JB):                      0.00630
Kurtosis:                       4.230   Cond. No.                         129.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [121]:
pd.set_option('display.max_columns', 1000)